# Model Evaluation

**Autores**: 

José Antonio Ruiz Heredia (josrui05@ucm.es) 

Néstor Marín 

**Descripción**: 

En este código realizamos la evaluación del modelo final.

Importamos las librerias y las rutas de nuestros datos.

In [2]:
import torch
import torchaudio
import torchaudio.transforms as T
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from torchvision.transforms import v2
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from torcheval.metrics import MulticlassAccuracy, MulticlassF1Score
from torch.optim.lr_scheduler import CosineAnnealingLR, ExponentialLR,CosineAnnealingWarmRestarts
import torch
import torch.nn as nn
import torchaudio
import torchaudio.functional as F
import random
from schedulefree import adamw_schedulefree, sgd_schedulefree
import pickle
import torch
from dvclive import Live

from torchaudio_augmentations import (
    Compose,
    LowPassFilter,
    HighLowPass,
    Noise,
    PitchShift,
    RandomApply,
)

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
import pandas as pd
import timm
import pickle
from sklearn.model_selection import StratifiedKFold

ModuleNotFoundError: No module named 'schedulefree'

In [ ]:

# # Suprimir todas las advertencias
# warnings.filterwarnings("ignore")

# # Suprimir una advertencia específica por su categoría
# warnings.filterwarnings("ignore", category=UserWarning)

# # Suprimir una advertencia específica por su mensaje
# warnings.filterwarnings("ignore", message="Some classes do not exist in the target")

print(torch.__version__)
print(torchaudio.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
#######################################################
#Audio hyper-param
# ESPEC = 'mfcc'
ESPEC = 'mel'
SAMPLE_RATE = 16000
MAX_DURATION = 8# 20
N_FFT = 1024
PRETRAINED = True
N = 30
if MAX_DURATION == 20:
    if N_FFT == 2048:
        TIME_MASK_PARAM =10
        FREQ_MASK_PARAM = 5
        HOP_LENGTH = 1024
    else:
        TIME_MASK_PARAM = 10
        FREQ_MASK_PARAM = 5
        HOP_LENGTH = 512
else:
    if N_FFT == 2048:
        TIME_MASK_PARAM = 20
        FREQ_MASK_PARAM = 5
        HOP_LENGTH = 1024
    else:
        TIME_MASK_PARAM = 40
        FREQ_MASK_PARAM = 5
        HOP_LENGTH = 512
        
N_MELS = 224 #128
N_MFCC = N_MELS
N_LFCC = N_MELS
RESIZE = False
IMG_SIZE = 224 #384, #300 , 240
NUM_CLASSES = 3
SAMPLE_RATE = 16000

num_samples = MAX_DURATION * SAMPLE_RATE

########################################################
#Training hyper-param
batch_size = 32
learning_rate = 1E-4
scheduler_learning = True
weight_decay = 1E-3

num_epochs = 50
early_stopping_patience = 10
label_smoothing = 0.1
STEP_TO_DECAY = 1
GAMMA_TO_DECAY = 0.95
WEIGHT = True
f_weighted = 1.5
AUG= False

HIDDEN_UNITS=[256,128, 64]
DROPOUT_RATE = 0.5
num_samples = MAX_DURATION * SAMPLE_RATE
MAX_PADDING = 118989


# exp_name = f"from_scratch_exp_{exp_num}"
params = {'epochs': num_epochs, 'lr': learning_rate, 'weight_decay': weight_decay, 
          'batch_size': batch_size, 'early_stopping_patience': early_stopping_patience,
          'label_smoothing': label_smoothing, 'num_classes': NUM_CLASSES, 'n_fft': N_FFT,
          'hop_length': HOP_LENGTH,'n_mels': N_MELS, 'max_duration': MAX_DURATION, 
          'exp_name': exp_name, 'exp_num': exp_num, 'n_layers': N, 'augmented': AUG, 'hidden_units': HIDDEN_UNITS, 
          'audios_max_duration': MAX_DURATION, 'espectogram':ESPEC, 'Freq_Mask': FREQ_MASK_PARAM, 
          'Time_Mask': TIME_MASK_PARAM, 'scheduler': scheduler_learning, 'dropout_rate': DROPOUT_RATE, 'model_name': MODEL_NAME
          , 'weighted': WEIGHT, 'img_size': IMG_SIZE, 'f_weighted': f_weighted}

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        self.audio_df = annotations_file
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.audio_df)

    def __getitem__(self, idx):
        audio = torchaudio.load(self.audio_df.iloc[idx, 0])
        label = self.audio_df.iloc[idx, 4]

        if self.transform:
            audio = self.transform(audio)
        if self.target_transform:
            label = self.target_transform(label)
        return audio, label


def preprocess_data(audio):
     # Load the audio file
  
    WAVEFORM, SAMPLE_RATE = audio
    padding_needed = (MAX_PADDING) - WAVEFORM.shape[1]
    padding = max(padding_needed, 0)
   
    waveform_padding = torch.nn.functional.pad(WAVEFORM, (0, padding))

    n_fft = N_FFT
    win_length = None
    hop_length = HOP_LENGTH
    n_mels = N_MELS
    
    mel_spectrogram = T.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=n_fft,
        win_length=win_length,
        hop_length=hop_length,
        center=True,
        pad_mode="reflect",
        power=2.0,
        norm="slaney",
        n_mels=n_mels,
        mel_scale="htk",
    )
    
    melspec = mel_spectrogram(waveform_padding)
    
    # log_mel_spectrogram= torchaudio.transforms.AmplitudeToDB(top_db=80)(melspec)
    log_mel_spectrogram= torchaudio.transforms.AmplitudeToDB(top_db=80)(melspec)
    if log_mel_spectrogram.max() - log_mel_spectrogram.min() != 0:
        log_mel_spectrogram_norm = (log_mel_spectrogram - log_mel_spectrogram.min()) / (log_mel_spectrogram.max() - log_mel_spectrogram.min())
    else:
        log_mel_spectrogram_norm = (log_mel_spectrogram - log_mel_spectrogram.min())  
    log_mel_spectrogram_norm =log_mel_spectrogram_norm[0]*255
    # log_mel_spectrogram_norm = torch.unsqueeze(log_mel_spectrogram_norm, 0)
    log_mel_spectrogram_norm_rgb = log_mel_spectrogram_norm.repeat(3, 1, 1)  # Repite el canal en las dimensiones de los canales
    
    if  RESIZE: 
        resize_image = transforms.Resize((IMG_SIZE, IMG_SIZE), interpolation=InterpolationMode.BICUBIC, max_size=None)
        log_mel_spectrogram_norm_rgb = resize_image(log_mel_spectrogram_norm_rgb)/255
    else:
        log_mel_spectrogram_norm_rgb = log_mel_spectrogram_norm_rgb/255
    log_mel_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    log_mel_imagenet = log_mel_normalize(log_mel_spectrogram_norm_rgb)
    return log_mel_imagenet


class MLPModel(torch.nn.Module):
    def __init__(self, input_size, hidden_units, dropout_rate):
        super(MLPModel, self).__init__()
        self.hidden_units = hidden_units
        self.dropout_rate = dropout_rate
        self.layers = torch.nn.ModuleList()
        
        # Agregar la primera capa oculta con la entrada original
        self.layers.append(torch.nn.Linear(input_size, hidden_units[0]))
        self.layers.append(torch.nn.BatchNorm1d(hidden_units[0]))
        self.layers.append(torch.nn.ReLU())
        self.layers.append(torch.nn.Dropout(dropout_rate))
        
        # Agregar el resto de las capas ocultas
        for i in range(len(hidden_units) - 1):
            self.layers.append(torch.nn.Linear(hidden_units[i], hidden_units[i+1]))
            self.layers.append(torch.nn.BatchNorm1d(hidden_units[i+1]))
            self.layers.append(torch.nn.ReLU())
            self.layers.append(torch.nn.Dropout(dropout_rate))
        self.layers.append(torch.nn.Linear(hidden_units[-1], NUM_CLASSES))
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class CustomConvNeXt(nn.Module):
    def __init__(self, N=0):
        super(CustomConvNeXt, self).__init__()
        
        # Cargar el modelo preentrenado
        self.pretrained_model  = timm.create_model(MODEL_NAME, pretrained=PRETRAINED, num_classes=0, global_pool='avg') #convnextv2_femto.fcmae_ft_in1k.fcmae_ft_in22k_in1k_384 
        #timm.create_model('convnextv2_nano.fcmae_ft_in22k_in1k', pretrained=True, num_classes=0)
        self.n_layers = N
        # Congelar todas las capas primero
        for name, param in self.pretrained_model.named_parameters():
           param.requires_grad = False

        # Descongelar las últimas 20 capas que no son BatchNormalization
        unfrozen_count = 0
        for name, param in reversed(list(self.pretrained_model.named_parameters())):
            if 'bn' not in name and unfrozen_count < self.n_layers:
                param.requires_grad = True
                unfrozen_count += 1
        
        
        #self.add_vit = self.pretrained_model.num_features
        self.additional_layer = MLPModel(self.pretrained_model.num_features, hidden_units=HIDDEN_UNITS, dropout_rate= DROPOUT_RATE)
        #self.additional_layer = torch.nn.Linear(self.pretrained_model.num_features, NUM_CLASSES)
    def forward(self, x):
        x = self.pretrained_model(x)
        #x = self.avgpool(x)
        #x = torch.flatten(x, 1)
        #x = self.fc(x)
        x = self.additional_layer(x)
        return x



                
# testing()


In [ ]:
df_audio_train = pd.read_parquet(f'/home/gass/audio-forgery-detection/data/train.parquet')
df_audio_test = pd.read_parquet(f'/home/gass/audio-forgery-detection/data/test.parquet')

print(f'Training shape: {df_audio_train.shape}, Testing shape: {df_audio_test.shape}')

ds_val_audios =  CustomImageDataset(df_audio_test, transform=preprocess_data)
dataloader_test = DataLoader(ds_val_audios, batch_size=batch_size, shuffle=False, num_workers=10)


In [ ]:
archivo_pickle = 'PicklesAndFinalModel/mel_preprocess_data_3_chans_history_torch_baseline_convnext_femto_224_custom_exp_2_mel_cv_aug_sin_resize.pkl'


# Cargar el archivo pickle
with open(archivo_pickle, 'rb') as archivo:
    datos = pickle.load(archivo)

# Utilizar los datos cargados
print(datos)

In [ ]:
model_path = 'PicklesAndFinalModel/test_mel_preprocess_data_3_chans_history_torch_baseline_convnext_femto_224_custom_exp_8_mel_test_sin_resize_mel_best_model.pth'
# Cargar el estado del modelo
modelo_estado = torch.load(model_path)
# Crear una instancia del modelo
model = CustomConvNeXt()  # Reemplaza "TuModelo" con el nombre de tu modelo

# Cargar el estado en el modelo
model.load_state_dict(modelo_estado['model_state_dict'])
model = model.to(device)
model.eval() # Set the model to evaluation mode
# # Hacer predicciones con el modelo
predict_list = []
predict_list_max = []
predicted_TH = []
import torch.nn.functional as F

with torch.no_grad():
    for inputs, labels in dataloader_test:
        inputs, labels = inputs.to(device), labels.to(device)
        # Make prediction
        outputs = model(inputs) #feed forward
        predict_list.extend(F.softmax(outputs, dim=1).cpu().numpy()) # apply softmax
        _, predicted = torch.max(outputs, 1)
        predict_list_max.extend(predicted.cpu().numpy())

In [ ]:
# predict_list_max_TH= []
# predicted = [1 if elements[1] > 0.5 else 0 for elements in predict_list]
# predict_list_max_TH.extend(predicted)

# Suponiendo que predict_list_max son tus predicciones y df_test_audios.label son las etiquetas verdaderas
predictions = predict_list_max#df_test_audios.predictions
true_labels = df_audio_test.label

# Genera el informe de clasificación
report = classification_report(true_labels, predictions, digits=3)

print(report)

# Genera la matriz de confusión
conf_mat = confusion_matrix(true_labels, predictions)
print(conf_mat)
# Importar la paleta de colores "Blues"
cmap = sns.color_palette("Blues")

# Definir etiquetas personalizadas para los ejes x e y
class_labels = ["Real", "Copy-move", "Splicing"]
conf_mat_percent = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis] * 100
# Visualizar la matriz de confusión en porcentajes con la paleta de colores "Blues"
plt.figure(figsize=(10, 7))
sns.heatmap(conf_mat_percent, annot=True, fmt='.2f', cmap=cmap, annot_kws={"size": 14})
plt.xlabel('Predicción', fontsize=16)
plt.ylabel('Etiqueta real', fontsize=16)
plt.xticks(ticks=[0.5, 1.5, 2.5], labels=class_labels, fontsize=12)
plt.yticks(ticks=[0.5, 1.5, 2.5], labels=class_labels, fontsize=12)
plt.show()